$$
\newcommand{\mat}[1]{\boldsymbol {#1}}
\newcommand{\mattr}[1]{\boldsymbol {#1}^\top}
\newcommand{\matinv}[1]{\boldsymbol {#1}^{-1}}
\newcommand{\vec}[1]{\boldsymbol {#1}}
\newcommand{\vectr}[1]{\boldsymbol {#1}^\top}
\newcommand{\rvar}[1]{\mathrm {#1}}
\newcommand{\rvec}[1]{\boldsymbol{\mathrm{#1}}}
\newcommand{\diag}{\mathop{\mathrm {diag}}}
\newcommand{\set}[1]{\mathbb {#1}}
\newcommand{\cset}[1]{\mathcal{#1}}
\newcommand{\norm}[1]{\left\lVert#1\right\rVert}
\newcommand{\pderiv}[2]{\frac{\partial #1}{\partial #2}}
\newcommand{\bb}[1]{\boldsymbol{#1}}
\newcommand{\E}[2][]{\mathbb{E}_{#1}\left[#2\right]}
\newcommand{\ip}[3]{\left<#1,#2\right>_{#3}}
\newcommand{\given}[]{\,\middle\vert\,}
\newcommand{\DKL}[2]{\cset{D}_{\text{KL}}\left(#1\,\Vert\, #2\right)}
\newcommand{\grad}[]{\nabla}
\newcommand{\norm}[1]{\left\lVert#1\right\rVert}
$$

# Part 4: Summary Questions
<a id=part4></a>

This section contains summary questions about various topics from the course material.

You can add your answers in new cells below the questions.

**Notes**

- Clearly mark where your answer begins, e.g. write "**Answer:**" in the beginning of your cell.
- Provide a full explanation, even if the question doesn't explicitly state so. We will reduce points for partial explanations!
- This notebook should be runnable from start to end without any errors.

### CNNs

1. Explain the meaning of the term "receptive field" in the context of CNNs.

**Answer:** 

Receptive field is the region in the input space which produces the feature in the following layers of the CNN.

2. Explain and elaborate about three different ways to control the rate at which the receptive field grows from layer to layer. Compare them to each other in terms of how they combine input features.

**Answer:** 

1. Increasing the number of convolutional layers. **Each extra layer increases the receptive field size by the kernel size.**

2. Adding pooling layers, which also reduce the dimensions of the feature maps. Thus, it reduces the number of parameters to learn and the amount of computation performed in the network. The pooling layer summarises the features present in a region of the feature map generated by a convolution layer. **They increases the receptive field size multiplicatively.**

3. Dilated convolutions. They introduce spacing between the values of a convolutional kernel, the number of weights in the kernel is unchanged. **Increase the receptive field exponentially.**

3. Imagine a CNN with three convolutional layers, defined as follows:

In [1]:
import torch
import torch.nn as nn

cnn = nn.Sequential(
    nn.Conv2d(in_channels=3, out_channels=4, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(in_channels=4, out_channels=16, kernel_size=5, stride=2, padding=2),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(in_channels=16, out_channels=32, kernel_size=7, dilation=2, padding=3),
    nn.ReLU(),
)

cnn(torch.rand(size=(1, 3, 1024, 1024), dtype=torch.float32)).shape

torch.Size([1, 32, 122, 122])

What is the size (spatial extent) of the receptive field of each "pixel" in the output tensor?

**Answer:**

Using the fromulas:

$$ r_{out} = r_{in}+((k_{in} - 1) * \prod_{i=1}^{k-1}s_i) $$

Kernels with dilation:

$$ k_{prev} = r * (k - 1) + 1 $$

We will get:

Layer 1 (Conv2D): $$ k = 3, s = 1 $$
$$ R_1 = 1 + (3 - 1)*1 = 3 $$

Layer 2 (Pooloing): 
$$ R_2 = R_1 + (2 - 1) * 2 = 5 $$

Layer 3 (Conv2D): 
$$ R_3 = R_2 + (5 - 1) * 2^2 = 21 $$

Layer 4 (Pooloing): 
$$ R_4 = R_3 + (2 - 1) * 2^3 = 29 $$

Layer 5 (Conv2D): 
$$ R_5 = R_4 + (13 - 1) * 2^3 = 125 $$

The size of the receptive field of each "pixel" in the output tensor is [125 x 125]

4. You have trained a CNN, where each layer $l$ is represented by the mapping $\vec{y}_l=f_l(\vec{x};\vec{\theta}_l)$, and $f_l(\cdot;\vec{\theta}_l)$ is a convolutional layer (not including the activation function).

  After hearing that residual networks can be made much deeper, you decide to change each layer in your network you used the following residual mapping instead $\vec{y}_l=f_l(\vec{x};\vec{\theta}_l)+\vec{x}$, and re-train.

  However, to your surprise, by visualizing the learned filters $\vec{\theta}_l$ you observe that the original network and the residual network produce completely different filters. Explain the reason for this.

**Answer:**

In residual networks we use skip connections to sum the output of a layer with the input of another deeper layer (after skiping few layers), this allows to create deeper networks and solves the problem of vanishing gradients. Optimization process is different, it results in different filters.

### Dropout

1. **True or false**: dropout must be placed only after the activation function.

**Answer:** 

False. It doesn't metter where it placed since it sets a fraction of units to be zero.

2. After applying dropout with a drop-probability of $p$, the activations are scaled by $1/(1-p)$. Prove that this scaling is required in order to maintain the value of each activation unchanged in expectation.

**Answer:** 

When we apply the dropout only a fraction of the neurons is activated during training, while during the test we will activate all of them, thats why we need to do scaling to compensate.

### Losses and Activation functions

1. You're training a an image classifier that, given an image, needs to classify it as either a dog (output 0) or a hotdog (output 1). Would you train this model with an L2 loss? if so, why? if not, demonstrate with a numerical example. What would you use instead?

**Answer:**

The default loss function used for classification task is binary cross-entropy, which maximizes the likelihood of classification. L2 loss measures the squered error between the prediction and the label, it's the default loss function for regression tasks.

$L_2 loss = \sum_{i=0}^{N} (y_i-y_i^{pred})^2$


$BCE loss = \frac{1}{N} \sum_{i=i}^{N} -(y_i\cdot log(p_i) + (1-y_i)\cdot log(1-p_i))$



2. After months of research into the origins of climate change, you observe the following result:

<center><img src="https://sparrowism.soc.srcf.net/home/piratesarecool4.gif" /></center>

You decide to train a cutting-edge deep neural network regression model, that will predict the global temperature based on the population of pirates in `N` locations around the globe.
You define your model as follows:

In [ ]:
import torch.nn as nn

N = 42  # number of known global pirate hot spots
H = 128
mlpirate = nn.Sequential(
    nn.Linear(in_features=N, out_features=H),
    nn.Sigmoid(),
    *[
        nn.Linear(in_features=H, out_features=H),
        nn.Sigmoid(),
    ]*N,
    nn.Linear(in_features=H, out_features=1),
)

While training your model you notice that the loss reaches a plateau after only a few iterations.
It seems that your model is no longer training.
What is the most likely cause?

**Answer:**

The model is 42 layers deep and has no skip connections, also it uses sigmoid activation, which is good only for the final layer. The model is no longer training due to the vanishing gradients.

3. Referring to question 2 above: A friend suggests that if you replace the `sigmoid` activations with `tanh`, it will solve your problem. Is he correct? Explain why or why not.

**Answer:**

The range of a sigmoid is [0 ,1], the range of a tanh is [-1 ,1], it can help a little, but the model is so deep that it doesn't look like this increase can solve the problem of vanishing gradients.

4. Regarding the ReLU activation, state whether the following sentences are **true or false** and explain:
    A. In a model using exclusively ReLU activations, there can be no vanishing gradients.
    
    
    
**Answer: True.** But we still can get zero-nodes.
    
    
    
    B. The gradient of ReLU is linear with its input when the input is positive.
    
    
    
**Answer: False.** The gradient is constant and equals to 1.
    
    
    
    C. ReLU can cause "dead" neurons, i.e. activations that remain at a constant value of zero.
    
    
    
**Answer: True.** For negative values it will be zero.
    

### Optimization

1. Explain the difference between: stochastic gradient descent (SGD), mini-batch SGD and regular gradient descent (GD).

**Answer:** 
In GD the whole dataset used to calculate the average loss and make an update to the weights.

In Mini-batch SGD the loss calculation and weights update are made for a fraction of a dataset at each time. It takes one epoch to go over whole training set.

Stochastic gradient descent (SGD) calculates gradient for one point and backpropogates.


2. Regarding SGD and GD:
    1. Provide at least two reasons for why SGD is used more often in practice compared to GD.
    2. In what cases can GD not be used at all?

**Answer:**

A. 1 - Memory is limited. 2 - Can stuck at local minimum.

B. When the dataset is too big.

3. You have trained a deep resnet to obtain SoTA results on ImageNet.
While training using mini-batch SGD with a batch size of $B$, you noticed that your model converged to a loss value of $l_0$ within $n$ iterations (batches across all epochs) on average.
Thanks to your amazing results, you secure funding for a new high-powered server with GPUs containing twice the amount of RAM.
You're now considering to increase the mini-batch size from $B$ to $2B$.
Would you expect the number of of iterations required to converge to $l_0$ to decrease or increase when using the new batch size? explain in detail.

**Answer:** 

It's difficult to know. On the one hand, we expect the number of iterations to decrease, because in each batch we are averaging over more samples, on the other hand too large batch size can lead to poor generalization.

4. For each of the following statements, state whether they're **true or false** and explain why.
    1. When training a neural network with SGD, every epoch we perform an optimization step for each sample in our dataset.
    1. Gradients obtained with SGD have less variance and lead to quicker convergence compared to GD.
    1. SGD is less likely to get stuck in local minima, compared to GD.
    1. Training  with SGD requires more memory than with GD.
    1. Assuming appropriate learning rates, SGD is guaranteed to converge to a local minimum, while GD is guaranteed to converge to the global minimum.
    1. Given a loss surface with a narrow ravine (high curvature in one direction): SGD with momentum will converge more quickly than Newton's method which doesn't have momentum.

**Answer:**

A. True. For every update we consider one sample.

B. False. Since we calculate for one sample at a time gradients are more affected by noise.

C. True. There is no chance that every sample in SGD will lead to the same local minimum.

D. False. In SGD we need a memomry to store only one sample. 

E. False. We can't guarantee that.

F. False. Even though momentum prevents from SGD to oscilate in a narrow ravine, but in Newton's method the second derivative improves convergence more effectively.


5. In tutorial 5 we saw an example of bi-level optimization in the context of deep learning, by embedding an optimization problem as a layer in the network.
    1. **True or false**: In order to train such a network, the inner optimization problem must be solved with a descent based method (such as SGD, LBFGS, etc).
  Provide a mathematical justification for your answer.

**Answer:** 

False: In tutorial we saw that there are cases when minimum can be found without using descent minimum, by analytical solution.



6. You have trained a neural network, where each layer $l$ is represented by the mapping $\vec{y}_l=f_l(\vec{x};\vec{\theta}_l)$ for some arbitrary parametrized functions $f_l(\cdot;\vec{\theta}_l)$.
  Unfortunately while trying to break the record for the world's deepest network, you discover that you are unable to train your network with more than $L$ layers.
    1. Explain the concepts of "vanishing gradients", and "exploding gradients".
    2. How can each of these problems be caused by increased depth?
    3. Provide a numerical example demonstrating each.
    4. Assuming your problem is either of these, how can you tell which of them it is without looking at the gradient tensor(s)?

**Answer:**

A. Vanishing gradients happens when icreasingly small gradients backpropogate through the network for the update. Activation functions with plateu like sigmoid and tanh may lead to this problem. When we multiply low values by the chain rule multiple times the gradient becomes zero. Exploding gradients caused by very large derivative, the model becomes unstable.

B. Due to the chain rule.

C. If we assume 3 layer CNN, by the chain rule, for the first layer:

$\frac{d(f(f(f(x)))}{d(x)} = \frac{d(f(f(f(x)))}{d(f(f(x)))} \cdot \frac{df(f(x))}{d(f(x))} \cdot \frac{df(x)}{d(x))}$

If activation function is a power of high or low order, the gradints will explode or vanish respectively.

D. The loss will reach the plateau if the gradients are vanishing and oscilate if the gradients are exploding.


### Backpropagation

1. You wish to train the following 2-layer MLP for a binary classification task:
  $$
  \hat{y}^{(i)} =\mat{W}_2~ \varphi(\mat{W}_1 \vec{x}^{(i)}+ \vec{b}_1) + \vec{b}_2
  $$
  Your wish to minimize the in-sample loss function is defined as
  $$
  L_{\mathcal{S}} = \frac{1}{N}\sum_{i=1}^{N}\ell(y^{(i)},\hat{y}^{(i)}) + \frac{\lambda}{2}\left(\norm{\mat{W}_1}_F^2 + \norm{\mat{W}_2}_F^2 \right)
  $$
  Where the pointwise loss is binary cross-entropy:
  $$
  \ell(y, \hat{y}) =  - y \log(\hat{y}) - (1-y) \log(1-\hat{y})
  $$
  
  Write an analytic expression for the derivative of the final loss $L_{\mathcal{S}}$ w.r.t. each of the following tensors: $\mat{W}_1$, $\mat{W}_2$, $\mat{b}_1$, $\mat{b}_2$, $\mat{x}$.

2. Given the following code snippet, implement the custom backward function `part4_affine_backward` in `hw4/answers.py` so that it passes the `assert`s.

In [ ]:
from torch.autograd import Function

from hw4.answers import part4_affine_backward

N, d_in, d_out = 100, 11, 7
dtype = torch.float64
X = torch.rand(N, d_in, dtype=dtype)
W = torch.rand(d_out, d_in, requires_grad=True, dtype=dtype)
b = torch.rand(d_out, requires_grad=True, dtype=dtype)

def affine(X, W, b):
    return 0.5 * X @ W.T + b

class AffineLayerFunction(Function):
    @staticmethod
    def forward(ctx, X, W, b):
        result = affine(X, W, b)
        ctx.save_for_backward(X, W, b)
        return result

    @staticmethod
    def backward(ctx, grad_output):
        return part4_affine_backward(ctx, grad_output)

l1 = torch.sum(AffineLayerFunction.apply(X, W, b))
l1.backward()
W_grad1 = W.grad
b_grad1 = b.grad

l2 = torch.sum(affine(X, W, b))
W.grad = b.grad = None
l2.backward()
W_grad2 = W.grad
b_grad2 = b.grad

assert torch.allclose(W_grad1, W_grad2)
assert torch.allclose(b_grad1, b_grad2)

### Sequence models

1. Regarding word embeddings:
    1. Explain this term and why it's used in the context of a language model.
    1. Can a language model like the sentiment analysis example from the tutorials be trained without an embedding (i.e. trained directly on sequences of tokens)? If yes, what would be the consequence for the trained model? if no, why not?

**Answer:** 

A. Word embeddings is a way to represent words as a tensor while the words that are close in the tensor space are expected to have similar meaning. It's used in language model to allow words to be processed by it and perform calculations.

B. No, because we need a way to make numerical calculations.

2. Considering the following snippet, explain:
    1. What does `Y` contain? why this output shape?
    2. How you would implement `nn.Embedding` yourself using only torch tensors. 

In [ ]:
import torch.nn as nn

X = torch.randint(low=0, high=42, size=(5, 6, 7, 8))
embedding = nn.Embedding(num_embeddings=42, embedding_dim=42000)
Y = embedding(X)
print(f"{Y.shape=}")

**Answer:** 

A. Y contains embeddings of size of X with extra dimension of 42000.

B. We can write: embedding = torch.rand(size=(num_embeddings, embedding_dim)) and then: Y = torch.gather(input=embedding, dim=0, index=X).

3. Regarding truncated backpropagation through time (TBPTT) with a sequence length of $S$: State whether the following sentences are **true or false**, and explain.
    1. TBPTT uses a modified version of the backpropagation algorithm.
    2. To implement TBPTT we only need to limit the length of the sequence provided to the model to length $S$.
    3. TBPTT allows the model to learn relations between input that are at most $S$ timesteps apart.

**Answer:**

A. True. Herea losses are accumulated, and then the update made by using the accumulated gradients from all timesteps.

B. False. Input remains the same, the sequence for backpropogation changes.

C. False. We can learn relations between input that are more than S timesteps, we keep the hidden state of the previous sequence, which used then in the next sequence, so the output will depend on all timesteps.

### Attention

1. In tutorial 7 (part 2) we learned how to use attention to perform alignment between a source and target sequence in machine translation.
    1. Explain qualitatively what the addition of the attention mechanism between the encoder and decoder does to the hidden states that the encoder and decoder each learn to generate (for their language). How are these hidden states different from the model without attention?
  
  2. After learning that self-attention is gaining popularity thanks to the shiny new transformer models, you decide to change the model from the tutorial: instead of the queries being equal to the decoder hidden states, you use self-attention, so that the keys, queries and values are all equal to the encoder's hidden states (with learned projections). What influence do you expect this will have on the learned hidden states?


### Unsupervised learning

1. As we have seen, a variational autoencoder's loss is comprised of a reconstruction term and  a KL-divergence term. While training your VAE, you accidentally forgot to include the KL-divergence term.
What would be the qualitative effect of this on:

    1. Images reconstructed by the model during training ($x\to z \to x'$)?
    1. Images generated by the model ($z \to x'$)?

2. Regarding VAEs, state whether each of the following statements is **true or false**, and explain:
    1. The latent-space distribution generated by the model for a specific input image is $\mathcal{N}(\vec{0},\vec{I})$.
    2. If we feed the same image to the encoder multiple times, then decode each result, we'll get the same reconstruction.
    3. Since the real VAE loss term is intractable, what we actually minimize instead is it's upper bound, in the hope that the bound is tight.

2. Regarding GANs, state whether each of the following statements is **true or false**, and explain:
    1. Ideally, we want the generator's loss to be low, and the discriminator's loss to be high so that it's fooled well by the generator.
    2. It's crucial to backpropagate into the generator when training the discriminator.
    3. To generate a new image, we can sample a latent-space vector from $\mathcal{N}(\vec{0},\vec{I})$.
    4. It can be beneficial for training the generator if the discriminator is trained for a few epochs first, so that it's output isn't arbitrary.
     5. If the generator is generating plausible images and the discriminator reaches a stable state where it has 50% accuracy (for both image types), training the generator more will further improve the generated images.

### Graph Neural Networks

1. You have implemented a graph convolutional layer based on the following formula, for a graph with $N$ nodes:
$$
\mat{Y}=\varphi\left( \sum_{k=1}^{q} \mat{\Delta}^k \mat{X} \mat{\alpha}_k + \vec{b} \right).
$$
    1. Assuming $\mat{X}$ is the input feature matrix of shape $(N, M)$: what does $\mat{Y}$ contain in it's rows?
    1. Unfortunately, due to a bug in your calculation of the Laplacian matrix, you accidentally zeroed the row and column $i=j=5$ (assume more than 5 nodes in the graph).
What would be the effect of this bug on the output of your layer, $\mat{Y}$?

2. We have discussed the notion of a Receptive Field in the context of a CNN. How would you define a similar concept in the context of a GCN (i.e. a model comprised of multiple graph convolutional layers)?